# Tactical Insights API Endpoint Testing

This notebook tests the API endpoints for the Tactical Insights visualization category. We'll use the `requests` library to make HTTP calls to our locally running FastAPI server.

## Setup
First, let's set up our environment and import necessary libraries:

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from IPython.display import JSON
import os
import sys
import networkx as nx

# Add the root directory to the path so we can import app modules
sys.path.append('/workspaces/football-insights-api')

# Base URL for our API
BASE_URL = "http://localhost:8000/api"

# Helper function to make API calls
def call_api(endpoint, params=None):
    """Make an API call to the specified endpoint with optional parameters"""
    url = f"{BASE_URL}/{endpoint}"
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: Status code {response.status_code}")
        print(response.text)
        return None

# Helper to check if the API is running
def is_api_running():
    try:
        response = requests.get(f"{BASE_URL.split('/api')[0]}/")
        return response.status_code == 200
    except requests.ConnectionError:
        return False

# Check if API is running
if not is_api_running():
    print("API not running. Please start it manually with:")
    print("uvicorn app.main:app --host 0.0.0.0 --port 8000")
else:
    print("API is running!")

## Test Pass Network Endpoint

Let's test the pass network endpoint, which provides data for visualizing team structure and connections:

In [ ]:
# Call the pass network endpoint
# Using a sample match and team
pass_network = call_api("tactics/pass-network", params={
    "match_id": 3835324,  # Example match ID (Sweden Women's vs Netherlands Women's)
    "team_id": 858,       # Example team ID (Sweden Women's)
    "min_passes": 3
})

# Display the raw data
if pass_network:
    print("API Response Overview:")
    print(f"Match ID: {pass_network['match_id']}")
    print(f"Team: {pass_network['team_name']}")
    print(f"Players in network: {len(pass_network['network']['players'])}")
    print(f"Pass connections: {len(pass_network['network']['connections'])}")
    print("\nStructural Analysis:")
    print(f"Key players: {', '.join([p['name'] for p in pass_network['analysis']['key_players']])}")
    print(f"Team width: {pass_network['analysis']['team_width']:.2f} meters")
    print(f"Team depth: {pass_network['analysis']['team_depth']:.2f} meters")
    print(f"Connection density: {pass_network['analysis']['connection_density']:.2f}")
    
    # Visualize the pass network
    plt.figure(figsize=(12, 8))
    
    # Simple football pitch drawing function
    def draw_pitch(ax):
        # Pitch dimensions in StatsBomb data: 120x80
        pitch_length = 120
        pitch_width = 80
        
        # Main pitch outline
        ax.plot([0, 0], [0, pitch_width], 'black')
        ax.plot([0, pitch_length], [pitch_width, pitch_width], 'black')
        ax.plot([pitch_length, pitch_length], [pitch_width, 0], 'black')
        ax.plot([pitch_length, 0], [0, 0], 'black')
        
        # Middle line
        ax.plot([pitch_length/2, pitch_length/2], [0, pitch_width], 'black')
        
        # Set pitch appearance
        ax.set_xlim([-5, pitch_length + 5])
        ax.set_ylim([-5, pitch_width + 5])
        ax.set_facecolor('#74a9cf')
        ax.set_aspect('equal')
    
    # Set up figure and axis
    fig, ax = plt.subplots(figsize=(12, 8))
    draw_pitch(ax)
    
    # Create a graph using networkx
    G = nx.DiGraph()
    
    # Add nodes
    player_positions = {}
    for player in pass_network['network']['players']:
        player_id = player['player_id']
        pos = (player['avg_x'], player['avg_y'])
        G.add_node(player_id, pos=pos, name=player['name'], position=player['position'])
        player_positions[player_id] = pos
    
    # Add edges
    for conn in pass_network['network']['connections']:
        G.add_edge(conn['source'], conn['target'], weight=conn['passes'], success_rate=conn['success_rate'])
    
    # Get position dictionary for networkx
    pos = nx.get_node_attributes(G, 'pos')
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, 
                          node_size=300, 
                          node_color='red',
                          alpha=0.8,
                          ax=ax)
    
    # Draw player labels
    labels = {node: G.nodes[node]['name'] for node in G.nodes()}
    nx.draw_networkx_labels(G, pos, labels=labels, font_size=8, ax=ax)
    
    # Draw edges with width based on pass count
    edges = G.edges()
    weights = [G[u][v]['weight'] * 0.5 for u, v in edges]
    nx.draw_networkx_edges(G, pos, 
                          edgelist=edges,
                          width=weights,
                          edge_color='black',
                          alpha=0.5,
                          arrows=True,
                          arrowsize=10,
                          ax=ax)
    
    plt.title(f"Pass Network: {pass_network['team_name']}")
    plt.tight_layout()
    plt.show()
else:
    print("Failed to get pass network data.")

## Test PPDA Analysis Endpoint

Now let's test the PPDA analysis endpoint, which provides data on pressing intensity:

In [ ]:
# Call the PPDA analysis endpoint
ppda_analysis = call_api("tactics/ppda", params={
    "competition_id": 53,  # Example competition ID (UEFA Women's Euro)
    "team_id": 858,        # Example team ID (Sweden Women's)
    "match_id": None,      # Get data for all matches
    "opposition_half_only": True
})

# Display the raw data
if ppda_analysis:
    print("API Response:")
    JSON(ppda_analysis)
    
    # Create a visualization of the PPDA values
    if ppda_analysis.get('ppda_by_match'):
        plt.figure(figsize=(12, 6))
        
        # Extract data
        match_ids = [m['match_id'] for m in ppda_analysis['ppda_by_match']]
        ppda_values = [m['ppda'] for m in ppda_analysis['ppda_by_match']]
        opponents = [m['opponent'] for m in ppda_analysis['ppda_by_match']]
        
        # Create bar chart
        bars = plt.bar(range(len(match_ids)), ppda_values, color='skyblue')
        
        # Add average line
        plt.axhline(y=ppda_analysis['average_ppda'], color='red', linestyle='-', 
                   label=f"Avg PPDA: {ppda_analysis['average_ppda']:.2f}")
        
        # Add data labels
        for i, bar in enumerate(bars):
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                   f'{height:.2f}',
                   ha='center', va='bottom', rotation=0)
        
        plt.xlabel('Match')
        plt.ylabel('PPDA (lower = more intense pressing)')
        plt.title(f"PPDA Analysis: {ppda_analysis['team_name']}")
        plt.xticks(range(len(match_ids)), opponents, rotation=45, ha='right')
        plt.legend()
        plt.tight_layout()
        plt.show()
    
        # Add interpretation
        print("\nInterpretation:")
        print(f"The average PPDA for {ppda_analysis['team_name']} is {ppda_analysis['average_ppda']:.2f}.")
        print("Lower PPDA values indicate more aggressive pressing (fewer passes allowed before a defensive action).")
        
        # Find matches with most and least intense pressing
        if ppda_values:
            min_ppda_idx = ppda_values.index(min(ppda_values))
            max_ppda_idx = ppda_values.index(max(ppda_values))
            
            print(f"\nMost intense pressing: Against {opponents[min_ppda_idx]} (PPDA: {ppda_values[min_ppda_idx]:.2f})")
            print(f"Least intense pressing: Against {opponents[max_ppda_idx]} (PPDA: {ppda_values[max_ppda_idx]:.2f})")
else:
    print("Failed to get PPDA analysis data.")

## Test Shot Creation Analysis Endpoint

Finally, let's test the shot creation analysis endpoint, which provides data on shot creation patterns:

In [ ]:
# Call the shot creation analysis endpoint
shot_creation = call_api("tactics/shot-creation", params={
    "match_id": 3835324,  # Example match ID
    "team_id": 858        # Example team ID
})

# Display the raw data
if shot_creation:
    print("API Response Overview:")
    print(f"Team: {shot_creation['team_name']}")
    print(f"Total shots: {shot_creation['total_shots']}")
    print(f"Shots from key passes: {shot_creation['key_pass_shots']} ({shot_creation['key_pass_percentage']:.1f}%)")
    
    print("\nShot Creation Zones:")
    for zone, count in shot_creation['creation_zones'].items():
        print(f"{zone}: {count}")
    
    # Visualize shot creation
    plt.figure(figsize=(12, 8))
    
    # Simple football pitch drawing function
    def draw_pitch(ax):
        # Pitch dimensions in StatsBomb data: 120x80
        pitch_length = 120
        pitch_width = 80
        
        # Main pitch outline
        ax.plot([0, 0], [0, pitch_width], 'black')
        ax.plot([0, pitch_length], [pitch_width, pitch_width], 'black')
        ax.plot([pitch_length, pitch_length], [pitch_width, 0], 'black')
        ax.plot([pitch_length, 0], [0, 0], 'black')
        
        # Middle line and boxes
        ax.plot([pitch_length/2, pitch_length/2], [0, pitch_width], 'black')
        
        # Add thirds
        ax.plot([40, 40], [0, pitch_width], 'black', alpha=0.2)
        ax.plot([80, 80], [0, pitch_width], 'black', alpha=0.2)
        
        # Add vertical channels
        ax.plot([0, pitch_length], [pitch_width/3, pitch_width/3], 'black', alpha=0.2)
        ax.plot([0, pitch_length], [2*pitch_width/3, 2*pitch_width/3], 'black', alpha=0.2)
        
        # Set pitch appearance
        ax.set_xlim([-5, pitch_length + 5])
        ax.set_ylim([-5, pitch_width + 5])
        ax.set_facecolor('#74a9cf')
        ax.set_aspect('equal')
    
    # Set up figure and axis
    fig, ax = plt.subplots(figsize=(12, 8))
    draw_pitch(ax)
    
    # Plot key pass locations and connections to shots
    for kp in shot_creation['key_pass_locations']:
        if isinstance(kp['location'], list) and isinstance(kp['end_location'], list):
            # Plot key pass location
            pass_x, pass_y = kp['location'][0], kp['location'][1]
            shot_x, shot_y = kp['end_location'][0], kp['end_location'][1]
            
            # Determine color based on shot outcome
            color = 'green' if kp['resulting_shot_outcome'] == 'Goal' else 'red'
            
            # Plot the key pass location
            ax.scatter(pass_x, pass_y, color='blue', s=50, alpha=0.7)
            
            # Plot the shot location
            ax.scatter(shot_x, shot_y, color=color, s=100, alpha=0.7, 
                      marker='*' if kp['resulting_shot_outcome'] == 'Goal' else 'o')
            
            # Draw arrow from key pass to shot
            ax.arrow(pass_x, pass_y, shot_x-pass_x, shot_y-pass_y, 
                    color='black', width=0.3, head_width=2, alpha=0.5)
    
    # Add legend
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Key Pass'),
        plt.Line2D([0], [0], marker='*', color='w', markerfacecolor='green', markersize=10, label='Goal'),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='No Goal')
    ]
    ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, 0.05), ncol=3)
    
    plt.title(f"Shot Creation Analysis: {shot_creation['team_name']}")
    
    # Add zone labels
    ax.text(20, 40, f"Def Third\n({shot_creation['creation_zones']['defensive_third']})", ha='center', va='center', bbox=dict(facecolor='white', alpha=0.7))
    ax.text(60, 40, f"Mid Third\n({shot_creation['creation_zones']['middle_third']})", ha='center', va='center', bbox=dict(facecolor='white', alpha=0.7))
    ax.text(100, 40, f"Final Third\n({shot_creation['creation_zones']['final_third']})", ha='center', va='center', bbox=dict(facecolor='white', alpha=0.7))
    
    plt.tight_layout()
    plt.show()
    
    # Create zone distribution visualization
    plt.figure(figsize=(12, 5))
    
    # Horizontal zones
    plt.subplot(1, 2, 1)
    horizontal_data = [
        shot_creation['creation_zones']['defensive_third'],
        shot_creation['creation_zones']['middle_third'], 
        shot_creation['creation_zones']['final_third']
    ]
    plt.pie(horizontal_data, 
           labels=['Defensive Third', 'Middle Third', 'Final Third'],
           autopct='%1.1f%%',
           startangle=90,
           colors=['#ff9999', '#66b3ff', '#99ff99'])
    plt.title('Shot Creation by Horizontal Zone')
    
    # Vertical zones
    plt.subplot(1, 2, 2)
    vertical_data = [
        shot_creation['creation_zones']['left_channel'],
        shot_creation['creation_zones']['central_channel'], 
        shot_creation['creation_zones']['right_channel']
    ]
    plt.pie(vertical_data, 
           labels=['Left Channel', 'Central Channel', 'Right Channel'],
           autopct='%1.1f%%',
           startangle=90,
           colors=['#c2c2f0', '#ffcc99', '#99e6e6'])
    plt.title('Shot Creation by Vertical Channel')
    
    plt.tight_layout()
    plt.show()
else:
    print("Failed to get shot creation analysis data.")

## Conclusion

In this notebook, we've tested three key API endpoints for tactical analysis:

1. **Pass Network** - Visualizing team structure and connections between players
2. **PPDA Analysis** - Measuring pressing intensity across matches
3. **Shot Creation** - Analyzing patterns in how shots are created

These endpoints provide rich data for tactical visualizations and analysis in the Football Insights platform. Next steps could include:

1. Implementing additional tactical metrics like build-up speed, transition analysis, or defensive line height
2. Creating more sophisticated visualizations with D3.js or other frontend libraries
3. Integrating these insights with player-specific analysis
4. Developing comparison visualizations between teams or matches